<a href="https://colab.research.google.com/github/GabrielPazz/ImersaoIA-Alura/blob/main/Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes_(Final).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [5]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição será realizada é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho no Instagram, LinkedIn e outras plataformas onde a Alura costuma divulgar seus eventos e cursos.
*   **Assinar a newsletter da Alura:** Cadastre seu e-mail no site da Alura para receber informações sobre lançamentos e eventos.
*   **Consultar o site da Alura:** Visite regularmente o site da Alura e procure na seção de cursos e eventos.

Assim que a data da próxima Imersão IA com Google Gemini for definida, a Alura certamente divulgará em seus canais de comunicação.

In [6]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá entre os dias 12 e 16 de maio de 2025. As inscrições para o curso, que é gratuito, podem ser feitas até o dia 11 de maio. A Imersão IA é um curso online com cinco aulas, onde você aprenderá a criar prompts, automatizar tarefas e desenvolver sistemas de agentes autônomos usando o Google Gemini. Ao final do curso, você receberá um certificado Alura + Google.


In [7]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura Imersão IA com Google Gemini']
Páginas utilizadas na resposta: alura.com.br, tecmundo.com.br, youtube.com



In [8]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [9]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [10]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [11]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [12]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [13]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [14]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [15]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [16]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: agentes de IA
Maravilha! Vamos então criar o post sobre novidades em agentes de IA

--- 📝 Resultado do Agente 1 (Buscador) ---



> Ok, aqui estão minhas buscas para encontrar os lançamentos mais recentes e relevantes sobre agentes de IA no último mês:
> 
> 
> Com base nas minhas buscas, aqui estão os lançamentos e tendências mais relevantes em agentes de IA para o último mês (meados de abril a meados de maio de 2025):
> 
> **1. Adoção Crescente de Agentes de IA em Empresas:**
> 
> *   A adoção de agentes de IA está acelerando em diversas empresas, transformando a gestão de processos. Empresas brasileiras também estão integrando essas ferramentas para simplificar tarefas, otimizar recursos e transformar a gestão de processos.
> *   Setores como e-commerce, atendimento ao cliente, marketing e finanças já incorporam agentes de IA, impulsionando a automação e a eficiência operacional.
> *   Espera-se que, até 2028, 33% dos aplicativos de software corporativo incluam IA Agentic, permitindo que 15% das decisões de trabalho do dia a dia sejam tomadas de forma autônoma.
> 
> **2. Novas Ferramentas e Plataformas para Agentes de IA:**
> 
> *   **IBM:** A IBM anunciou atualizações no watsonx Orchestrate, oferecendo soluções para integrar, customizar e implementar agentes, além de agentes pré-construídos para RH, vendas e compras. A IBM também lançou o Agent Catalog, que simplifica o desenvolvimento e implantação de agentes.
> *   **OpenAI:** A OpenAI está se destacando no mercado corporativo, com 32,4% das empresas nos EUA pagando por assinaturas de seus modelos e ferramentas de IA em abril de 2025.
> *   **Microsoft:** O Microsoft Copilot Studio permite que organizações criem agentes de IA de conversação personalizados, usando a IA generativa da Microsoft e conectando-se com o ecossistema Microsoft 365 e Azure.
> *   **NTT DATA:** Lançou o Smart AI AgentTM, um ecossistema empresarial com soluções específicas para diversos setores, além de uma solução patenteada que transforma bots legados em agentes inteligentes autônomos.
> 
> **3. Capacidades e Aplicações Aprimoradas:**
> 
> *   Os agentes de IA estão evoluindo para combinar maior poder de síntese de dados e capacidades aprimoradas de tomada de decisão, execução de tarefas e interação com múltiplas ferramentas com o mínimo de intervenção humana.
> *   Eles podem criar seus próprios prompts e resultados adicionais com base nas informações às quais têm acesso.
> *   Agentes de IA como o Claude da Anthropic podem determinar se uma pesquisa online pode aprimorar a resposta fornecida, garantindo informações atualizadas e com referências diretas às fontes originais.
> *   A Amazon está oferecendo uma experiência de codificação interativa, onde os desenvolvedores contam com um parceiro virtual que os auxilia em tempo real durante o processo de codificação, revisão de mudanças, criação de documentação e execução de testes.
> 
> **4. Tendências e Previsões:**
> 
> *   O Gartner apontou a Agentic AI como uma das principais tendências tecnológicas para 2025.
> *   Uma pesquisa global do Capgemini Research Institute indicou que 32% dos executivos de alto escalão colocam os agentes de IA como a principal tendência tecnológica em dados e IA para 2025.
> *   Espera-se que os agentes de IA "entrem no mercado de trabalho" e mudem materialmente os resultados das empresas.
> *   Em 2025, muita conversa será sobre traçar os limites do que os agentes têm permissão para fazer e não fazer, e sempre ter supervisão humana.
> 
> **5. Desafios e Considerações:**
> 
> *   A maioria das organizações ainda não está preparada para agentes de IA. É essencial expor as APIs que já existem nas empresas.
> *   É preciso submeter esses sistemas a testes rigorosos em ambientes controlados para evitar falhas, criar mecanismos para ações de rollback e garantir registros de auditoria.
> *   A segurança dos agentes será essencial, especialmente contra ataques como a prompt injection, que tenta manipular modelos para realizar ações indesejadas.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Com certeza! Analisando os lançamentos e tendências mais recentes em agentes de IA, o tema mais relevante para um post no momento é:
> 
> **Adoção Crescente de Agentes de IA em Empresas**
> 
> **Por que este tema é o mais relevante?**
> 
> *   **Atualidade e Impacto:** Reflete a rápida expansão da IA Agentic no mundo empresarial, com exemplos concretos de empresas brasileiras e globais adotando a tecnologia.
> *   **Interesse do Público:** A adoção em empresas é um tema que interessa tanto a profissionais de tecnologia quanto a líderes de negócios, pois mostra como a IA está transformando o mercado de trabalho e os resultados das empresas.
> *   **Tendências Futuras:** Aborda previsões sobre o crescimento contínuo da IA Agentic e sua influência na tomada de decisões autônomas, o que gera curiosidade e discussões sobre o futuro do trabalho.
> 
> **Pontos Mais Relevantes a Serem Abordados:**
> 
> 1.  **O que são Agentes de IA e como funcionam?** Uma breve explicação para o público que ainda não está familiarizado com o tema.
> 2.  **Adoção em Diversos Setores:** Detalhar como empresas de e-commerce, atendimento ao cliente, marketing e finanças estão utilizando agentes de IA para otimizar processos.
> 3.  **Exemplos de Empresas:** Mencionar exemplos de empresas que já estão colhendo os benefícios da IA Agentic, incluindo empresas brasileiras.
> 4.  **Ferramentas e Plataformas:** Apresentar as principais ferramentas e plataformas disponíveis no mercado, como IBM watsonx Orchestrate, Microsoft Copilot Studio e Smart AI AgentTM da NTT DATA.
> 5.  **Capacidades e Aplicações Aprimoradas:** Destacar como os agentes de IA estão evoluindo para combinar síntese de dados, tomada de decisão e interação com múltiplas ferramentas.
> 6.  **Tendências e Previsões:** Abordar as projeções do Gartner e do Capgemini Research Institute sobre o crescimento da IA Agentic e sua entrada no mercado de trabalho.
> 7.  **Desafios e Considerações:** Alertar sobre os desafios da implementação da IA Agentic, como a necessidade de preparar a infraestrutura, realizar testes rigorosos e garantir a segurança dos sistemas.
> 
> **Plano de Assuntos para o Post:**
> 
> 1.  **Título:** "Agentes de IA Transformam Empresas: Adoção, Tendências e Desafios"
> 2.  **Introdução:**
>     *   Contextualizar o crescimento da IA Agentic no mundo empresarial.
>     *   Apresentar o objetivo do post: explorar a adoção, tendências e desafios da IA Agentic.
> 3.  **O que são Agentes de IA?**
>     *   Explicar o conceito de agentes de IA e como eles diferem de outras formas de IA.
>     *   Mencionar suas principais características: autonomia, capacidade de aprendizado e interação com o ambiente.
> 4.  **Adoção em Diversos Setores:**
>     *   E-commerce: personalização da experiência do cliente, chatbots inteligentes.
>     *   Atendimento ao Cliente: resolução de problemas, suporte técnico.
>     *   Marketing: análise de dados, criação de campanhas personalizadas.
>     *   Finanças: detecção de fraudes, análise de riscos.
> 5.  **Exemplos de Empresas:**
>     *   Citar empresas que já estão utilizando agentes de IA com sucesso.
>     *   Incluir exemplos de empresas brasileiras que estão na vanguarda da IA Agentic.
> 6.  **Ferramentas e Plataformas:**
>     *   IBM watsonx Orchestrate: integração, customização e implementação de agentes.
>     *   Microsoft Copilot Studio: criação de agentes de conversação personalizados.
>     *   Smart AI AgentTM (NTT DATA): ecossistema empresarial com soluções específicas para diversos setores.
> 7.  **Capacidades e Aplicações Aprimoradas:**
>     *   Síntese de dados: capacidade de analisar grandes volumes de dados e extrair informações relevantes.
>     *   Tomada de decisão: capacidade de tomar decisões autônomas com base em dados e regras pré-definidas.
>     *   Interação com múltiplas ferramentas: capacidade de integrar-se com diferentes sistemas e aplicativos.
> 8.  **Tendências e Previsões:**
>     *   Gartner: Agentic AI como uma das principais tendências tecnológicas para 2025.
>     *   Capgemini Research Institute: executivos colocam agentes de IA como a principal tendência em dados e IA.
>     *   Expectativa de que agentes de IA "entrem no mercado de trabalho" e mudem os resultados das empresas.
> 9.  **Desafios e Considerações:**
>     *   Preparação da infraestrutura: necessidade de expor as APIs das empresas.
>     *   Testes rigorosos: importância de testar os sistemas em ambientes controlados.
>     *   Segurança: proteção contra ataques como a prompt injection.
> 10. **Conclusão:**
>     *   Reafirmar o potencial transformador da IA Agentic para as empresas.
>     *   Incentivar a discussão sobre os desafios e oportunidades da tecnologia.
>     *   Finalizar com uma chamada para ação: convidar os leitores a explorar mais sobre o tema e a considerar a adoção da IA Agentic em suas empresas.
> 


--------------------------------------------------------------


...

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-22' coro=<AsyncClient.aclose() done, defined at /usr/local/lib/python3.11/dist-packages/httpx/_client.py:1978> exception=RuntimeError('Event loop is closed')>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/httpx/_client.py", line 1985, in aclose
    await self._transport.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_transports/default.py", line 406, in aclose
    await self._pool.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 353, in aclose
    await self._close_connections(closing_connections)
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 345, in _close_connections
    await connection.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection.py", line 173, in aclose
    await self._connection.aclose()
  File "/usr/local/lib/pytho

--------------------------------------------------------------


Exception in thread Thread-14 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 138, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 126, in _invoke_run_async
    async for event in self.run_async(
  File "/u


--- 📝 Resultado do Agente 4 (Revisor) ---



--------------------------------------------------------------
